# Face recognition using LBPH

In [1]:
#importing necessary libraries
import cv2
import numpy as np
from datetime import datetime
import pywhatkit as kit
import smtplib, getpass
import glob
import os
import subprocess as sp

In [2]:
#function for cropping face and converting into gray scale

def img_crop_gray(model,img):
    
    f= model.detectMultiScale(img)
    
    if len(f)==0:
        #print('face not found')
        return None, None
    #crop image
    crop = img[f[0][1]: f[0][1]+f[0][2],f[0][0]:f[0][0]+f[0][3], : ]
    
    #convert all images to the same dimensions
    crop= cv2.resize(crop, (200,200))
    
    #convert the image to gray scale to reduce dimensionality
    crop= cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    
    return crop,f

In [ ]:
#collecting dataset

cam=cv2.VideoCapture(0)
model= cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
i=0
while i<100:
    ret, pic= cam.read()
    face,f=img_crop_gray(model,pic)
    
    if f is None:
        cv2.putText(pic,'face not found', (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('face recognition',pic)
        if cv2.waitKey(20)==13:
            break
            
    else:
        
        #save it to disk
        file_path= 'C:/Users/Anagha/Documents/mlops/cv/faces/taylor/'+str(i)+ '.jpg'  #selena
        cv2.imwrite(file_path, face)
        i=i+1
        
        #show bounding box around face
        cv2.rectangle(pic,(f[0][0],f[0][1]),(f[0][0]+f[0][3],f[0][1]+f[0][3]),[0,255,0], 3)
        
        #showing live count
        cv2.putText(face, str(i), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        
        #show the image
        cv2.imshow('face recognition',pic)
        cv2.imshow('crop', face)
        if cv2.waitKey(20)==13:
            break
            
            
cv2.destroyAllWindows()
cam.release()
print('dataset collected')

In [ ]:
#loading all image file names into a variable

file_loc_tay = os.path.join( 'faces','taylor', '*.jpg')
file_loc_sel= os.path.join( 'faces','selena', '*.jpg')

files_tay = glob.glob(file_loc_tay)
files_sel = glob.glob(file_loc_sel)

In [ ]:
files=files_tay

for i in files_sel:
    files.append(i)
print(len(files))

In [ ]:
images,names,labels=[],[],[]

In [ ]:
for i in files:
    p=cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    images.append(p)
    l=i.split('\\')
    names.append(l[1])

In [ ]:
images=np.asarray(images)

In [ ]:
for i in names:
    if i=='taylor':
        labels.append(0)
    else:
        labels.append(1)
labels=np.asarray(labels)

In [ ]:
#model creation
model_both=cv2.face_LBPHFaceRecognizer.create()
model_both.train(images,labels)

In [ ]:
def aws():
    # to launch instance
    # enter your specific details(image-id, subnet,security groups,keyname etc)
    os.system("aws ec2 run-instances --image-id <enter_here> --instance-type t2.micro --count 1 --subnet-id <enter_here> --tags Key=Face_recognition,Value=Myec2 --security-group-ids <enter_here> --key-name <enter_here>")
    print("launched")
    # to create volume
    os.system("aws ec2 create-volume --availability-zone <enter_here> --size 5 --tag-specifications ResourceType=volume,Tags={Key=Face_recognition,Value=Myebs}")
    print("volume created")
    print(sp.getoutput("aws ec2 describe-instances --filters \"Name=tag:Face_recognition,Values=Myec2\" --query \"Reservations[].Instances[].[InstanceId,LaunchTime]\""))
    print(sp.getoutput("aws ec2 describe-volumes --filters \"Name=tag:Face_recognition,Values=Myebs\" --query \"Volumes[].[VolumeId,CreateTime]\""))
    i_id = input("enter instance id:")
    v_id = input("enter volume id:")
    # to attach volume
    os.system("aws ec2 attach-volume --instance-id "+i_id+" --volume-id "+v_id+" --device /dev/xvdf" )
    print("attached")

In [ ]:
l=[("<enter_phone_number>", "<enter_mail_id>", "<enter_password>", "<enter_recipients_mail>")]

In [ ]:
#code for sending whatsapp msg and email

def send_email_msg(tup):
    ph,sender,pw,recipient= tup
    #whatsapp
    kit.sendwhatmsg(f"+91{ph}", "hey, a face has been detected!", datetime.now().hour, (datetime.now().minute) +1)
    print('whatsapp message sent')
    #gmail
    s= smtplib.SMTP('smtp.gmail.com', 587)
    
    #start TLS
    s.starttls()
    
    #authentication
    
    #sender login
    s.login(sender, pw)
    
    #message to be sent
    msg= 'hey a face has been detected!'
    
    #sending mail to recipient
    
    s.sendmail(sender, recipient, msg)
    
    #terminate session
    s.quit()
    
    print('email sent')

In [ ]:
#final working- model for 2 persons
det_msg=0
det_aws=0
cam=cv2.VideoCapture(0)
haarcascade= cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
    ret, pic= cam.read()
    face,f=img_crop_gray(haarcascade,pic)
    if f is None:
        
        cv2.putText(pic, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', pic )
        
        if cv2.waitKey(20)==13:
            break
            
    
    else:
        
        result= model_both.predict(face)
        #print(result)
        
        if result[1] < 500:
            confidence = int( 100 * (1 - (result[1])/400) )
    
    
        if confidence > 85:
            #show bounding box around face
            cv2.rectangle(pic,(f[0][0],f[0][1]),(f[0][0]+f[0][3],f[0][1]+f[0][3]),[0,255,0], 3)
            if result[0]==0:
                cv2.putText(pic, "Hey taylor  confidence:"+str(confidence), (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                det_msg=1
            if result[0]==1:
                cv2.putText(pic, "Hey selena  confidence:"+str(confidence), (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                det_aws=1
            cv2.imshow('Face Recognition', pic )
            
        
        else:
            cv2.putText(pic, "who are you?", (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)        
            cv2.imshow('Face Recognition', pic )
        
        if cv2.waitKey(20)==13:
            break
    
            
cv2.destroyAllWindows()
cam.release()
print('done')

if det_msg==1:
    send_email_msg(l[0])
if det_aws==1:
    aws()